# Lab | API wrappers

In [114]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [12]:
# Getting the playlists from this website "The Best Spotify Playlists in 2021"
url = "https://audiohype.io/resources/the-best-spotify-playlists/"
response = requests.get(url)
response.status_code 

200

In [13]:
soup = BeautifulSoup(response.content, "html.parser")

In [41]:
# I read 29 playlists ids
playlists_ids = []
num_iter = len(soup.select("h3"))
for i in range(num_iter):
    temp = soup.select("h3 a")[i]["href"]
    playlists_ids.append(temp.split('playlist/')[1].split('?')[0])
playlists_ids


['37i9dQZF1DXcBWIGoYBM5M',
 '31ymdYCITDnZRtkKzP3Itp',
 '0cc8YMQWsSzODyTpdVB6mI',
 '37i9dQZF1DX5hHfOi73rY3',
 '37i9dQZF1DXa71eg5j9dKZ',
 '4kw9kdjzx1UmyWvpysl0y2',
 '1xNWydoMmA8210KeHU948w',
 '6vDGVr652ztNWKZuHvsFvx',
 '6J1r02xyO2qkMA9dDNZytJ',
 '5wtqmpRl17iVz2nW8U6njL',
 '0NCspsyf0OS4BsPgGhkQXM',
 '19y0UVk0bcrJWEqMwBHosj',
 '4oOZJEq1TBUti6PSouTo5M',
 '068WHS0zOWsqvn2uIBYb5D',
 '6ZTpgxK6BT92mmsqwETj9l',
 '37i9dQZF1DWSvKsRPPnv5o',
 '37i9dQZF1DXe4Cw8IKKIvr',
 '37i9dQZF1DXcDoDDetPsEg',
 '65xSncKQzG6Suseh5gfYP1',
 '0MSCX9tZWQmitMQsfhvZIl',
 '1PmwMQY86pJuAm7veFt3u2',
 '1cwmC5BQ6IIuLsbyZNBIJl',
 '6wjCvkAFovrVIRM8VfZLZG',
 '37i9dQZF1DWVV27DiNWxkR',
 '37i9dQZF1DWVzZlRWgqAGH',
 '37i9dQZF1DXc8kgYqQLMfH',
 '31FWVQBp3WQydWLNhO0ACi',
 '37i9dQZF1DXa9wYJr1oMFq',
 '37i9dQZF1DX8FwnYE6PRvL']

In [1]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

In [91]:
def get_name_artists_from_track(track):
    return [(track["name"],artist["name"],track["uri"],sp.audio_features(track["uri"])) for artist in track["artists"]]

def get_name_artists_from_playlist_item(playlist_item):
    return get_name_artists_from_track(playlist_item['track'])

def get_name_artists_from_playlist(playlist):
    return list(map(get_name_artists_from_playlist_item, playlist['items']))

def flatten(input_list):
    return [item for sublist in input_list for item in sublist]

In [115]:
# Reading all playlists and collecting song, artist and audio features

from random import randint
from time import sleep

all_songs = []
for i in range(len(playlists_ids)):
        playlist = sp.user_playlist_tracks("spotify", playlists_ids[i])
        try:
            all_songs += (flatten(get_name_artists_from_playlist(playlist)))
        except:
            print("Playlist: ", playlists_ids[i], "<-- There is an error with this playlist:")
        sleep(randint(1,3))

Playlist:  5wtqmpRl17iVz2nW8U6njL <-- There is an error with this playlist:


In [116]:
len(all_songs)

3202

In [135]:
all_songs[0]

('Oh My God',
 'Adele',
 'spotify:track:3Kkjo3cT83cw09VJyrLNwX',
 [{'danceability': 0.534,
   'energy': 0.725,
   'key': 1,
   'loudness': -3.906,
   'mode': 1,
   'speechiness': 0.046,
   'acousticness': 0.091,
   'instrumentalness': 0,
   'liveness': 0.0264,
   'valence': 0.55,
   'tempo': 88.098,
   'type': 'audio_features',
   'id': '3Kkjo3cT83cw09VJyrLNwX',
   'uri': 'spotify:track:3Kkjo3cT83cw09VJyrLNwX',
   'track_href': 'https://api.spotify.com/v1/tracks/3Kkjo3cT83cw09VJyrLNwX',
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3Kkjo3cT83cw09VJyrLNwX',
   'duration_ms': 225149,
   'time_signature': 4}])

In [137]:
import pickle

with open("all_songs.txt", "wb") as f:   #Pickling
    pickle.dump(all_songs, f)